<a href="https://colab.research.google.com/github/LuigiSigillo/nlp2021-hw/blob/master/hw1/stud/nlp_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First approach (word-level)


### Imports

In [4]:
from google.colab import drive
# general
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
from typing import *

# torch
import torch
import json
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD

# NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

drive.mount('/content/drive')
root_folder = '/content/drive/My Drive/NLP/nlp2021-hw1'
dataset_folder = os.path.join(root_folder,'data')

''' code to download and move the glove embeddings in the right folder '''
#! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
#! unzip -d data/glove.6B
#! cd '/content/drive/My Drive/NLP/nlp2021-hw1'
#!unzip '/content/drive/My Drive/NLP/nlp2021-hw1/glove.6B.zip'
# !mv glove.6B.200d.txt '/content/drive/My Drive/NLP/nlp2021-hw1/model'
!ls "{root_folder}/model/"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
glove.6B.100d.txt  glove.6B.200d.txt  glove.6B.300d.txt  glove.6B.50d.txt


In [23]:
#@title Setup of hyperparameters{ run: "auto" }
WE_LENGTH = "50" #@param [50,100,200,300]
BATCH_SIZE = 32 #@param {type:"slider", min:8, max:64, step:8}


### Loading of Glove word embeddings

Added to the dictionary also the "UNK" and "SEP" words using a random vector

In [15]:
word_vectors = dict()
words_limit = 100_000 
with open(root_folder+'/model/glove.6B.'+WE_LENGTH+'d.txt') as f:

    next(f)  # skip header

    for i, line in tqdm(enumerate(f), total=words_limit):

        if i == words_limit:
            break

        word, *vector = line.strip().split(' ')
        vector = torch.tensor([float(c) for c in vector])
        
        word_vectors[word] = vector
# word_vectors["UNK"] = np.mean(np.array(list(word_vectors.values()), dtype=np.float64), axis=0)
word_vectors["UNK"] = torch.tensor(np.random.random(int(WE_LENGTH)),dtype=torch.float)
word_vectors["<SEP>"] = torch.tensor(np.random.random(int(WE_LENGTH)),dtype=torch.float)

Word-embedding-powered function $\phi$. Converts sentences to a vector by averaging the embeddings corresponding to each word in it

In [16]:
def phrase2vector(phrase: str, method: str, keyword: str) -> Optional[torch.Tensor]:
    #phrases_word_vector = [word_vectors[w] if w in word_vectors else word_vectors['UNK'] for w in phrase.split(' ')]
    # if len(phrases_word_vector) == 0:
    #     return None
    phrases_word_vector = []
    for w in phrase.split(' '):
        coeff = 1
        if w in word_vectors:
            if w == keyword:
                coeff = 1.5
            phrases_word_vector.append(word_vectors[w]*coeff)
        else:
            phrases_word_vector.append(word_vectors['UNK'])
    
    if len(phrases_word_vector) == 0:
        return None

    phrases_word_vector = torch.stack(phrases_word_vector)  # tensor shape: (#words X #features)
    if method=="avg":
        return torch.mean(phrases_word_vector, dim=0)
    elif method=="sum":
        return torch.sum(phrases_word_vector, dim=0)

### Dataset class and interfaces

In [17]:
class SentencesDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path: str, phrase2vector):
        self.data_store = []
        self.init_structures(dataset_path, phrase2vector)

    def init_structures(self, dataset_path: str, phrase2vector) -> None:

        with open(dataset_path) as f:
            for json_string in f:
                single_json = json.loads(json_string)
                keyword = single_json['sentence1'][int(single_json['start1']):int(single_json['end1'])]
                sentence =  self.remove_stopwords(single_json['sentence1']) + " <SEP> " + self.remove_stopwords(single_json['sentence2'])
                ground_t = np.float32(1) if single_json['label'] =='True' else np.float32(0)
                vector = phrase2vector(sentence,"avg",keyword)
                if vector is None:
                    continue
                self.data_store.append((vector,ground_t))
                



    def remove_stopwords(self,sent):
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(sent)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []

        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        
        return " ".join(filtered_sentence)
    
    def __len__(self) -> int:
        return len(self.data_store)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data_store[idx]

In [26]:
class SentencesDataModule(nn.Module):

    def __init__(
        self, 
        data_train_path: str,
        data_dev_path: str,
        batch_size: int,
        collate_fn=None
    ) -> None:
        super().__init__()
        self.data_train_path = data_train_path
        self.data_dev_path = data_dev_path
        self.batch_size = batch_size
        self.collate_fn = collate_fn

        self.train_dataset = None
        self.validation_dataset = None
        self.test_dataset = None

    def train_dataloader(self, *args, **kwargs) -> DataLoader:
        self.train_dataset = SentencesDataset(self.data_train_path, phrase2vector)
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        self.validation_dataset = SentencesDataset(self.data_dev_path, phrase2vector)
        return DataLoader(self.validation_dataset, batch_size=self.batch_size)


Loading and testing of the dataset

In [27]:
sentences_dm = SentencesDataModule(
    data_train_path=dataset_folder+'/train.jsonl',
    data_dev_path=dataset_folder+'/dev.jsonl',
    batch_size=BATCH_SIZE,
)
val_dataloader = sentences_dm.val_dataloader()
# print(word_vectors['test'])

for batch in val_dataloader:
    X, y = batch
    print(batch)
    print(f"batch X shape: {X.shape}")
    print(f"batch z shape: {y.shape}")
    break

[tensor([[ 0.2196, -0.0401, -0.1549,  ...,  0.3079, -0.0343,  0.1154],
        [ 0.2629, -0.1115, -0.0867,  ...,  0.4695,  0.1633,  0.1087],
        [ 0.3517,  0.2926, -0.0568,  ...,  0.1697,  0.0858,  0.0371],
        ...,
        [ 0.2190,  0.1479, -0.1112,  ..., -0.1255, -0.1117,  0.0582],
        [ 0.2601,  0.1581,  0.0031,  ...,  0.0359,  0.0391, -0.0203],
        [ 0.3069,  0.0681,  0.0119,  ..., -0.0385,  0.1100, -0.1006]]), tensor([0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])]
batch X shape: torch.Size([32, 50])
batch z shape: torch.Size([32])


### Training

Create the classifier class

In [29]:
class SentencesClassifier(nn.Module):

    def __init__(self, n_features: int, n_hidden: int):
        super().__init__()
        # classification function
        self.lin1 = torch.nn.Linear(n_features, n_hidden)
        self.output_layer = torch.nn.Linear(n_hidden, 1)
        
        # criterion
        self.loss_fn = torch.nn.BCELoss()
        

    def forward(self, x: torch.Tensor, y: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        # actual forward
        out = self.lin1(x)
        out = torch.relu(out)
        # compute logits (which are simply the out variable) and the actual probability distribution (pred, as it is the predicted distribution)
    
        logits = self.output_layer(out).squeeze(1)

        out = torch.sigmoid(logits)

        result = {'logits': logits, 'pred': out}

        # compute loss
        if y is not None:
            # torch optimizes its computation internally and takes as input the logits instead
            loss = self.loss(out, y)
            result['loss'] = loss

        return result

    def loss(self, pred, y):
        return self.loss_fn(pred, y)

Defining a trainer class to better separate our work.

In [30]:
class Trainer():
    def __init__(self, model, optimizer, device):

        self.device = device

        self.model = model
        self.optimizer = optimizer

        # starts requires_grad for all layers
        self.model.train()  # we are using this model for training (some layers have different behaviours in train and eval mode)
        self.model.to(self.device)  # move model to GPU if available

    def train(self, train_dataset, eval_dataset, epochs=1):

        train_loss = 0.0
        for epoch in tqdm(range(epochs)):
            epoch_loss = 0.0
            len_train = 0
            epoch_val_loss = 0.0
            len_val_train = 0
            accuracy = 0
            self.model.train()
            # each element (sample) in train_dataset is a batch
            for step, sample in enumerate(train_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                # outputs in the batch
                targets = sample[1].to(self.device)
                output_distribution = self.model(inputs)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss
                # calculates the gradient and accumulates
                loss.backward()  # we backpropagate the loss
                # updates the parameters
                self.optimizer.step()
                self.optimizer.zero_grad()

                epoch_loss += loss.item()
                len_train += 1
            
            self.model.eval()
            for step, sample in enumerate(eval_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                # outputs in the batch
                targets = sample[1].to(self.device)
                output_distribution = self.model(inputs)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss    
                
                accuracy += ((output_distribution['pred'] > 0.5) == targets).float().mean().item() #TODO
                epoch_val_loss += loss.item()
                len_val_train += 1
            
            avg_epoch_loss = epoch_loss / len_train
            avg_eval_loss = epoch_val_loss / len_val_train
            avg_accuracy_loss = accuracy / len_val_train
            print('Epoch: {} avg loss = {:0.4f} eval loss = {:0.4f} ACC = {:0.4f}'.format(epoch, avg_epoch_loss, avg_eval_loss, avg_accuracy_loss))

            train_loss += avg_epoch_loss
            
        torch.save(self.model.state_dict(),os.path.join(root_folder +"/model", 'state_{}.pt'.format(epoch)))  # save the model state

        avg_epoch_loss = train_loss / epochs
        return avg_epoch_loss

Instanciating the classifier and tune some hyperparameters such as the number of hidden layers learning rate and number of epochs

In [35]:
sent_classifier = SentencesClassifier(
    n_features=int(WE_LENGTH), 
    n_hidden=128 #@param {type:"slider", min:50, max:300, step:50}
)
learning_rate = 0.02 #@param {type:"slider", min:0.0001, max:0.1, step:0.001}
epochs = 150 #@param {type:"slider", min:50, max:300, step:10}

Defining the optimizer, we will use Stochastic gradient descent and instanciating the trainer to start the train.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.SGD(sent_classifier.parameters(), lr=learning_rate)

trainer = Trainer(sent_classifier, optimizer, device)

train_dataloader = sentences_dm.train_dataloader()

avg_loss = trainer.train(train_dataloader,val_dataloader, epochs=epochs)
print(avg_loss)

Method to run a prediction on a personal test phrase and see the prob of beeing of the same context

In [37]:
def predict(model, phrase2vector, phrase: str, keyword: str):
    phrase_vector = phrase2vector(phrase,"avg",keyword).to('cuda' if torch.cuda.is_available() else 'cpu')
    forward_out = model(phrase_vector.unsqueeze(0))  # add a dimension to create a one-item batch
    print(f"# Sentences: {phrase}")
    for i,prob in enumerate(forward_out["pred"]):
        print("\n {}".format( prob) )
predict(sent_classifier, phrase2vector, "The cat eats the mouse <SEP> Use the mouse to click on the button", "mouse")
predict(sent_classifier, phrase2vector, "The cat eats the mouse <SEP> The mouse escaped from the predator", "mouse")



# Sentences: The cat eats the mouse <SEP> Use the mouse to click on the button

 0.9253823757171631
# Sentences: The cat eats the mouse <SEP> The mouse escaped from the predator

 0.9676219820976257


# RNN

In [ ]:
word_index = dict()
vectors_store = []

# pad token, index = 0
vectors_store.append(torch.rand(WE_LENGTH))

# unk token, index = 1
vectors_store.append(torch.rand(WE_LENGTH))

for word, vector in word_vectors.items():
    word_index[word] = len(vectors_store)
    vectors_store.append(vector)

word_index = defaultdict(lambda: 1, word_index)  # default dict returns 1 (unk token) when unknown word
vectors_store = torch.stack(vectors_store)

In [ ]:
vocabulary_size, hidden_features = vectors_store.shape
print(f"Vocabulary size: {vocabulary_size}")
print(f"Hidden features: {hidden_features}")
word_index['pezzo']  # let's see if the word_index gives to us the unk index (1)


Vocabulary size: 100004
Hidden features: 100


1

In [ ]:
def review2indices(review: str,method: str,keyword: str) -> torch.Tensor:
    return torch.tensor([word_index[word] for word in review.split(' ')], dtype=torch.long)

In [ ]:
def rnn_collate_fn(
    data_elements: List[Tuple[torch.Tensor, torch.Tensor]] # list of (x, y) pairs
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:

    X = [de[0] for de in data_elements]  # list of index tensors

    # to implement the many-to-one strategy
    X_lengths = torch.tensor([x.size(0) for x in X], dtype=torch.long)

    X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=0)  #  shape (batch_size x max_seq_len)

    y = [de[1] for de in data_elements]
    y = torch.tensor(y)

    return X, X_lengths, y

In [ ]:
class AmazonReviewsRNNDataModule(nn.Module):

    def __init__(
        self, 
        data_train_path: str,
        data_dev_path: str,
        data_test_path: str,
        batch_size: int,
        collate_fn=None
    ) -> None:
        super().__init__()
        self.data_train_path = data_train_path
        self.data_dev_path = data_dev_path
        self.data_test_path = data_test_path
        self.batch_size = batch_size
        self.collate_fn = collate_fn

        self.train_dataset = None
        self.validation_dataset = None
        self.test_dataset = None

    def setup(self, stage: Optional[str] = None) -> None:
        if stage == 'fit':
            self.train_dataset = SentencesDataset(self.data_train_path, review2indices)
            self.validation_dataset = SentencesDataset(self.data_dev_path, review2indices)
        elif stage == 'test':
            self.test_dataset = SentencesDataset(self.data_test_path, review2indices)

    def train_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, collate_fn=rnn_collate_fn)

    def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.validation_dataset, batch_size=self.batch_size, collate_fn=rnn_collate_fn)

    def test_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.test_dataset, batch_size=self.batch_size, collate_fn=rnn_collate_fn)

In [ ]:
amazon_review_rnn_dm = AmazonReviewsRNNDataModule(
    data_train_path=dataset_folder+'/train.jsonl',
    data_dev_path=dataset_folder+'/dev.jsonl',
    data_test_path='data/dataset.test.tsv',
    batch_size=32,
)

In [ ]:
amazon_review_rnn_dm.setup('fit')

for batch in amazon_review_rnn_dm.val_dataloader():
    batch_X, batch_X_lengths, batch_y = batch
    print(batch_X)
    print(batch_X.shape)
    print(batch_y.shape)
    break

tensor([[    1,  8195,   954,  ...,  9580,     1,     3],
        [    1,  8195,   954,  ...,     0,     0,     0],
        [    1, 19372,   249,  ...,     0,     0,     0],
        ...,
        [    1,   360,  2604,  ...,     0,     0,     0],
        [    1,   750,  5518,  ...,     0,     0,     0],
        [    1,   750,  5518,  ...,     0,     0,     0]])
torch.Size([32, 44])
torch.Size([32])


In [ ]:
class AmazonReviewRecurrentClassifier(nn.Module):

    def __init__(
        self,
        vectors_store: torch.Tensor,
        n_hidden: int
    ) -> None:
        super().__init__()

        # embedding layer
        self.embedding = torch.nn.Embedding.from_pretrained(vectors_store)

        # recurrent layer
        self.rnn = torch.nn.LSTM(input_size=vectors_store.size(1), hidden_size=n_hidden, num_layers=1, batch_first=True)

        # classification head
        self.lin1 = torch.nn.Linear(n_hidden, n_hidden)
        self.lin2 = torch.nn.Linear(n_hidden, 1)

        # criterion
        self.loss_fn = torch.nn.BCELoss()
        self.device = 'cuda'

    def forward(
        self, 
        X: torch.Tensor, 
        X_length: torch.Tensor, 
        y: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:

        # embedding words from indices
        embedding_out = self.embedding(X)

        # recurrent encoding
        recurrent_out = self.rnn(embedding_out)[0]
        
        # here we utilize the sequences length to retrieve the last token 
        # output for each sequence
        batch_size, seq_len, hidden_size = recurrent_out.shape

        # we flatten the recurrent output
        # now I have a long sequence of batch x seq_len vectors 
        flattened_out = recurrent_out.reshape(-1, hidden_size)
        
        # and we use a simple trick to compute a tensor of the indices 
        # of the last token in each batch element
        last_word_relative_indices = X_length - 1
        # tensor of the start offsets of each element in the batch
        sequences_offsets = torch.arange(batch_size, device=self.device) * seq_len
        # e.g. (0, 5, 10, 15, ) + ( 3, 2, 1, 4 ) = ( 3, 7, 11, 19 )
        summary_vectors_indices = sequences_offsets + last_word_relative_indices

        # finaly we retrieve the vectors that should summarize every review.
        # (i.e. the last token in the sequence)
        summary_vectors = flattened_out[summary_vectors_indices]

        # now we can classify the reviews with a feedforward pass on the summary
        # vectors
        out = self.lin1(summary_vectors)
        out = torch.relu(out)
        out = self.lin2(out).squeeze(1)

        # compute logits (which are simply the out variable) and the actual probability distribution (pred, as it is the predicted distribution)
        logits = out
        pred = torch.softmax(logits, dim=-1)

        result = {'logits': logits, 'pred': pred}

        # compute loss
        if y is not None:
            # while mathematically the CrossEntropyLoss takes as input the probability distributions,
            # torch optimizes its computation internally and takes as input the logits instead
            loss = self.loss(logits, y)
            result['loss'] = loss

        return result

    def loss(self, pred, y):
        return self.loss_fn(pred, y)

In [ ]:
class Trainer():
    def __init__(self, model, optimizer, device):

        self.device = device

        self.model = model
        self.optimizer = optimizer

        # starts requires_grad for all layers
        self.model.train()  # we are using this model for training (some layers have different behaviours in train and eval mode)
        self.model.to(self.device)  # move model to GPU if available

    def train(self, train_dataset, eval_dataset, epochs=1):

        train_loss = 0.0
        for epoch in tqdm(range(epochs)):
            epoch_loss = 0.0
            len_train = 0
            epoch_val_loss = 0.0
            len_val_train = 0
            accuracy = 0
            self.model.train()
            # each element (sample) in train_dataset is a batch
            for step, sample in enumerate(train_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                x_lenghts = sample[1].to(self.device)
                # outputs in the batch
                targets = sample[2].to(self.device)
                output_distribution = self.model(inputs, x_lenghts)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss
                # calculates the gradient and accumulates
                loss.backward()  # we backpropagate the loss
                # updates the parameters
                self.optimizer.step()
                self.optimizer.zero_grad()

                epoch_loss += loss.item()
                len_train += 1
                
            self.model.eval()
            for step, sample in enumerate(eval_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                x_lenghts = sample[1].to(self.device)
                # outputs in the batch
                targets = sample[2].to(self.device)
                output_distribution = self.model(inputs, x_lenghts)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss    
                
                accuracy += ((output_distribution['pred'] > 0.5) == targets).float().mean().item() #TODO
                epoch_val_loss += loss.item()
                len_val_train += 1
            
            avg_epoch_loss = epoch_loss / len_train
            avg_eval_loss = epoch_val_loss / len_val_train
            avg_accuracy_loss = accuracy / len_val_train
            print('Epoch: {} avg loss = {:0.4f} eval loss = {:0.4f} ACC = {:0.4f}'.format(epoch, avg_epoch_loss, avg_eval_loss, avg_accuracy_loss))

            train_loss += avg_epoch_loss
            # torch.save(self.model.state_dict(),
            #            os.path.join(output_folder, 'state_{}.pt'.format(epoch)))  # save the model state

        avg_epoch_loss = train_loss / epochs
        return avg_epoch_loss

In [ ]:
amazon_review_recurrent_classifier = AmazonReviewRecurrentClassifier(vectors_store, n_hidden=128)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.Adam(amazon_review_recurrent_classifier.parameters(), lr=0.001)
trainer = Trainer(amazon_review_recurrent_classifier, optimizer, device)


train_dataloader = amazon_review_rnn_dm.train_dataloader()
avg_loss = trainer.train(train_dataloader,amazon_review_rnn_dm.val_dataloader(), epochs=150)